In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#matplotlib inline
from __future__ import print_function
import keras
from keras.utils import to_categorical
import os
from keras.preprocessing.image import ImageDataGenerator, load_img

In [ ]:
from keras.applications import VGG19 # VGG19

vgg_conv = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

In [ ]:
vgg_conv.summary()

In [ ]:
train_dir = './ALL_Leukemia_dataset/CNN_transfer_UnSegmented/Training' # Path of training folder
validation_dir = './ALL_Leukemia_dataset/CNN_transfer_UnSegmented/Testing' #Path of testing folder

nTrain = 208 # Number of images in the training folder
nVal = 52 # Number of images in the testing folder

In [ ]:
###### Feature extration from the training folder ######

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

train_features = np.zeros(shape=(nTrain, 7, 7, 512))
train_labels = np.zeros(shape=(nTrain,2))

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nTrain:
        break
        
train_features = np.reshape(train_features, (nTrain, 7 * 7 * 512))

In [ ]:
###### Feature extration from the testing folder ######

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

validation_features = np.zeros(shape=(nVal, 7, 7, 512))
validation_labels = np.zeros(shape=(nVal,2))

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224), #InceptionV3 229, 229
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

i = 0
for inputs_batch, labels_batch in validation_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    validation_features[i * batch_size : (i + 1) * batch_size] = features_batch
    validation_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nVal:
        break

validation_features = np.reshape(validation_features, (nVal, 7 * 7 * 512))

In [ ]:
### Export Training\Testing image features as npy file to Matlab

import numpy as np
np.save("train_features.npy", train_features)
np.save("train_labels.npy", train_labels)
np.save("validation_features.npy", validation_features)
np.save("validation_labels.npy", validation_labels)